In [2]:
import numpy as np
import pandas as pd

In [3]:
data_path = "/home/alan-k/workspace/fork/openai-api/attentionx-icl/poc/openai_api/ickd__gpt-3-5-turbo__sst2___n67349_s0_e67349.csv"
df = pd.read_csv(data_path)
df.head()

,index,sentence,label,positive_prob,negative_prob
0,0,hide new secretions from the parental units,0,0.001140,0.998400
1,1,"contains no wit , only labored gags",0,-1.000000,0.994419
2,2,that loves its characters and communicates som...,1,0.999983,0.000013
3,3,remains utterly satisfied to remain the same t...,0,0.016052,0.982627
4,4,on the worst revenge-of-the-nerds clichés the ...,0,0.000070,0.999721


In [4]:
pos_null_df = df[(df["positive_prob"] == -1) & (df["negative_prob"] != -1)]
neg_null_df = df[(df["positive_prob"] != -1) & (df["negative_prob"] == -1)]
both_null_df = df[(df["positive_prob"] == -1) & (df["negative_prob"] == -1)]
not_null_df = df[(df["positive_prob"] != -1) & (df["negative_prob"] != -1)]

print(f"total: {len(df)}")
print(f"positive null: {len(pos_null_df) / len(df):.2f} ({len(pos_null_df)}/{len(df)})")
print(f"negative null: {len(neg_null_df) / len(df):.2f} ({len(neg_null_df)}/{len(df)})")
print(f"both null: {len(both_null_df) / len(df):.2f} ({len(both_null_df)}/{len(df)})")
print(f"not null: {len(not_null_df) / len(df):.2f} ({len(not_null_df)}/{len(df)})")

total: 67349
positive null: 0.12 (8182/67349)
negative null: 0.01 (887/67349)
both null: 0.01 (397/67349)
not null: 0.86 (57883/67349)


In [12]:
cleaned_df = not_null_df

print(f"cleaned: {len(cleaned_df) / len(df):.2f} ({len(cleaned_df)}/{len(df)})")

cleaned: 0.86 (57883/67349)


In [5]:
imputed_df = df[(df["positive_prob"] != -1) | (df["negative_prob"] != -1)]
for i, row in imputed_df.iterrows():
    if row["positive_prob"] == -1:
        imputed_df.at[i, "positive_prob"] = 1 - row["negative_prob"]
    elif row["negative_prob"] == -1:
        imputed_df.at[i, "negative_prob"] = 1 - row["positive_prob"]

print(f"imputed: {len(imputed_df) / len(df):.2f} ({len(imputed_df)}/{len(df)})")

imputed: 0.99 (66952/67349)


In [9]:
def calculate_accuracy(df):
    prob = df["positive_prob"]
    pred = (prob > 0.5).astype(int)
    label = df["label"]
    return np.mean(pred == label)

def calculate_precision(df):
    prob = df["positive_prob"]
    pred = (prob > 0.5).astype(int)
    label = df["label"]
    tp = np.sum(pred & label)
    fp = np.sum(pred & ~label)
    return tp / (tp + fp)

def calculate_recall(df):
    prob = df["positive_prob"]
    pred = (prob > 0.5).astype(int)
    label = df["label"]
    tp = np.sum(pred & label)
    fn = np.sum(~pred & label)
    return tp / (tp + fn)


def print_metrics(df):
    print(f"accuracy: {calculate_accuracy(df):.2f}")
    print(f"precision: {calculate_precision(df):.2f}")
    print(f"recall: {calculate_recall(df):.2f}")

In [10]:
print_metrics(cleaned_df)

accuracy: 0.93
precision: 0.97
recall: 0.91


In [11]:
print_metrics(imputed_df)

accuracy: 0.93
precision: 0.97
recall: 0.90
